In [ ]:
%pip install git+https://github.com/huggingface/transformers.git
%pip install Pillow redvid
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


In [ ]:
import requests
from redvid import Downloader
import subprocess
from PIL import Image

video_input_file = 'video.mp4'
video_output_file = 'video.jpg'

img_output_file = 'image.jpg'

def downloadRedditVideo(url):
  reddit = Downloader(max_q=True)
  reddit.log = False
  reddit.url = url
  reddit.path = "./"
  reddit.filename = video_output_file
  reddit.download()
  print("Downloaded " + video_output_file)

def extractFrameFromVideo():
  input_file = video_input_file
  output_file = video_output_file

  #delete video.jpg if it exists
  subprocess.call(['rm', '-f', output_file])

  duration = float(subprocess.check_output(['ffprobe', '-i', input_file, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=%s' % ("p=0")]).strip())
  middle_time = duration / 2
  #extract the frame
  subprocess.call(['ffmpeg', '-i', input_file, '-ss', str(middle_time), '-vframes', '1', '-q:v', '2', output_file], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
  print("Extracted frame from " + input_file + " to " + output_file)

  image = Image.open(output_file)
  return image


def generate_blip2(image):
  inputs = processor(image, return_tensors="pt").to(device, torch.float16)
  generated_ids = model.generate(**inputs, max_new_tokens=20)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
  print(generated_text)



## load subreddits

In [ ]:
import glob

json_files = glob.glob('./data/*.json')
print(json_files)

import json

def getRedditSubreddits(json_files):
  subreddits = {}
  for json_file in json_files:
    with open(json_file) as f:
      data = json.load(f)
      firstKey = next(iter(data))
      subredditName = data[firstKey]['subreddit']
      subreddits[subredditName] = data
  return subreddits

def saveredditSubreddits(subreddits):
  for subreddit in subreddits:
    with open('./data/' + subreddit + '_top_posts.json', 'w') as outfile:
      json.dump(subreddits[subreddit], outfile, indent=4)

subreddits = getRedditSubreddits(json_files)

print("subreddits", subreddits.keys())


In [ ]:
def downloadImage(url):
  img_data = requests.get(url).content
  with open('image_name.jpg', 'wb') as handler:
      handler.write(img_data)

def downloadRedditMedia(domain, url):
  

In [8]:


mediaDomains = ['i.redd.it', 'v.redd.it', 'i.imgur.com', 'youtu.be', 'youtube.com', 'i.reddituploads.com']
mediaDomains = {
    "i.redd.it": downloadImage,
    "i.reddituploads.com": downloadImage,
    "v.redd.it": downloadRedditVideo,
    "i.imgur.com": downloadImage,
    "youtu.be": downloadYoutubeThumbnail,
    "youtube.com": downloadYoutubeThumbnail,
  }
def isMediaDomain(url):
  for mediaDomain in mediaDomains.keys():
    if mediaDomain in url or url.startswith('self.'):
      return True
  return False

for subreddit_name in subreddits:
  subreddit = subreddits[subreddit_name]
  print("subreddit", subreddit_name)
  for post_id in subreddit:
    post = subreddit[post_id]
    isValidUrlPost = post['url'] != None and post['text'] == None and isMediaDomain(post['url'])
    if not isValidUrlPost:
      continue

    #print url
    print(post['url'])



NameError: name 'downloadImage' is not defined